In [7]:
import functools
import os
from typing import List, Sequence

from eval import interpolator as interpolator_lib
from eval import util
from absl import logging
import apache_beam as beam
import mediapy as media
import natsort
import numpy as np
import tensorflow as tf
from tqdm.auto import tqdm

# Controls TF_CCP log level.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

C:\ProgramData\Anaconda3\envs\frame-interpolation\lib\site-packages\apache_beam\__init__.py:79: UserWarning: This version of Apache Beam has not been sufficiently tested on Python 3.9. You may encounter bugs or missing features.
  warnings.warn(
C:\ProgramData\Anaconda3\envs\frame-interpolation\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def _output_frames(frames: List[np.ndarray], frames_dir: str):
  """Writes PNG-images to a directory.

  If frames_dir doesn't exist, it is created. If frames_dir contains existing
  PNG-files, they are removed before saving the new ones.

  Args:
    frames: List of images to save.
    frames_dir: The output directory to save the images.

  """
  if tf.io.gfile.isdir(frames_dir):
    old_frames = tf.io.gfile.glob(f'{frames_dir}/frame_*.png')
    if old_frames:
      logging.info('Removing existing frames from %s.', frames_dir)
      for old_frame in old_frames:
        tf.io.gfile.remove(old_frame)
  else:
    tf.io.gfile.makedirs(frames_dir)
  for idx, frame in tqdm(
      enumerate(frames), total=len(frames), ncols=100, colour='green'):
    util.write_image(f'{frames_dir}/frame_{idx:03d}.png', frame)
  logging.info('Output frames saved in %s.', frames_dir)

class ProcessDirectory(beam.DoFn):
  """DoFn for running the interpolator on a single directory at the time."""

  def setup(self):
    self.interpolator = interpolator_lib.Interpolator(
        modelpath, align,
        [bh, bw])

    if outputvideo:
      ffmpeg_path = util.get_ffmpeg_path()
      media.set_ffmpeg(ffmpeg_path)

  def process(self, directory: str):
    input_frames_list = [
        natsort.natsorted(tf.io.gfile.glob(f'{directory}/*{ext}'))
        for ext in input_ext
    ]
    input_frames = functools.reduce(lambda x, y: x + y, input_frames_list)
    logging.info('Generating in-between frames for %s.', directory)
    frames = list(
        util.interpolate_recursively_from_files(
            input_frames, time2inter, self.interpolator))
    _output_frames(frames, f'{directory}')

    if outputvideo:
      media.write_video(f'{directory}/interpolated.mp4', frames, fps=fps)
      logging.info('Out;put video saved at %s/interpolated.mp4.', directory)

In [ ]:
if __name__=='__main__':
    pattern = r"\\shelter\Kyu\motility_interpolation\filmtest_train_skip5\sequences\*\*"
    directories = tf.io.gfile.glob(pattern)
    directories = directories[:3]
    print(directories)

    modelpath=r"\\shelter\Kyu\motility_interpolation\pretrained_models\film_net\Style\saved_model"
    time2inter=5

    align=64
    bh=1
    bw=1
    outputvideo=False
    fps=30
    input_ext=['1.png','3.png']

    pipeline = beam.Pipeline('DirectRunner')
    (pipeline | 'Create directory names' >> beam.Create(directories)  # pylint: disable=expression-not-assigned
       | 'Process directories' >> beam.ParDo(ProcessDirectory()))
    result = pipeline.run()
    result.wait_until_finish()

In [8]:
pattern = r"\\shelter\Kyu\motility_interpolation\filmtest_train_skip5\sequences\*\*"
directories = tf.io.gfile.glob(pattern)
directories

['\\\\shelter\\Kyu\\motility_interpolation\\filmtest_train_skip5\\sequences\\xy005\\000',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest_train_skip5\\sequences\\xy005\\001',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest_train_skip5\\sequences\\xy005\\002',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest_train_skip5\\sequences\\xy005\\003',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest_train_skip5\\sequences\\xy005\\004',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest_train_skip5\\sequences\\xy005\\005',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest_train_skip5\\sequences\\xy005\\006',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest_train_skip5\\sequences\\xy005\\007',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest_train_skip5\\sequences\\xy005\\008',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest_train_skip5\\sequences\\xy005\\009',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest_train_skip5\\sequences\\xy005\\010',
 '\\\\shel

In [12]:
input_ext=['1.png','3.png']
directory=directories[0]
input_frames_list = [
    natsort.natsorted(tf.io.gfile.glob(f'{directory}/*{ext}'))
    for ext in input_ext
]
input_frames_list

[['\\\\shelter\\Kyu\\motility_interpolation\\filmtest_train_skip5\\sequences\\xy005\\000\\im1.png'],
 ['\\\\shelter\\Kyu\\motility_interpolation\\filmtest_train_skip5\\sequences\\xy005\\000\\im3.png']]

In [14]:
input_frames = functools.reduce(lambda x, y: x + y, input_frames_list)
logging.info('Generating in-between frames for %s.', directory)
input_frames

['\\\\shelter\\Kyu\\motility_interpolation\\filmtest_train_skip5\\sequences\\xy005\\000\\im1.png',
 '\\\\shelter\\Kyu\\motility_interpolation\\filmtest_train_skip5\\sequences\\xy005\\000\\im3.png']